In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[4],3b
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004}"
nv,-,1000
tsfc,300,300


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312        1  76.905931                    1  54.636468
0.000750        2  41.034808                    2  41.629973
0.001052        3  33.260413                    3  36.060567
0.001476        4  26.397651                    4  29.531671
0.002070        5  20.664477                    5  22.680369
0.002904        6  16.081574                    6  16.382875
0.004074        7  12.530568                    7  11.550211
0.005714        8   9.827322                    8   8.493279
0.008015        9   7.771220                    9   6.801676
0.011243       10   6.181154                   10   5.730989
0.015771       11   4.904023                   11   4.713219
0.022122       12   3.810494                   12   3.608858
0.031031       13   2.745643                   13   2.393335
0.043528       14   1.400430                   14   0.993240
0.061057       15   0.979717                   15   0.634814
0.085645       16   1.753881                   16   1.561722
0.120136       17   2.556847                   17   2.561087
0.168516       18   3.178320                   18   3.354221
0.236378       19   3.834086                   19   4.048589
0.331549       20   4.631986                   20   4.674031
0.465100       21   5.785579                   21   5.572409
0.652400       22   7.479591                   22   7.164416
0.915100       23   7.610555                   23   7.464781
1.283650       24   6.458223                   24   6.413784
1.800600       25   5.480241                   25   5.304717
2.525700       26   4.431571                   26   4.166466
3.542800       27   3.639498                   27   3.595424
4.969550       28   3.061406                   28   3.194562
6.970850       29   2.593042                   29   2.695491
9.778100       30   2.180979                   30   2.198914
13.715850      31   1.799921                   31   1.826838
19.239350      32   1.443614                   32   1.415960
26.987250      33   1.121902                   33   1.091698
37.855300      34   0.874494                   34   0.933756
53.100050      35   0.507419                   35   0.568248
73.887500      36   0.035067                   36   0.021156
97.662500      37  -0.288973                   37  -0.322882
121.437500     38  -0.218950                   38  -0.220198
145.212500     39   0.021281                   39   0.036728
168.987500     40   0.076195                   40   0.083312
192.762500     41   0.100852                   41   0.103433
216.537500     42   0.113216                   42   0.113669
240.312500     43   0.119040                   43   0.116707
264.087500     44   0.120925                   44   0.114658
287.862500     45   0.120223                   45   0.110067
311.637500     46   0.117680                   46   0.104668
335.412500     47   0.113842                   47   0.099031
359.187500     48   0.108994                   48   0.094918
382.962500     49   0.103216                   49   0.090335
406.737500     50   0.097257                   50   0.086832
430.512500     51   0.091236                   51   0.083775
454.287500     52   0.085324                   52   0.079365
478.062500     53   0.079579                   53   0.076469
501.837500     54   0.074098                   54   0.071550
525.612500     55   0.069003                   55   0.068774
549.387500     56   0.064263                   56   0.063503
573.162500     57   0.060032                   57   0.059607
596.937500     58   0.056680                   58   0.055386
620.712500     59   0.054813                   59   0.052826
644.487500     60   0.056110                   60   0.053591
668.262500     61   0.061919                   61   0.058971
692.037500     62   0.069189                   62   0.065952
715.812500     63   0.067589                   63

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -16.203643   0.000000 -16.203643 -15.792515 -1.311255e-07   
0.000624    2     -16.204011   0.006056 -16.197955 -15.792912  4.437956e-03   
0.000876    3     -16.204178   0.007445 -16.196733 -15.793078  5.844328e-03   
0.001229    4     -16.204387   0.009044 -16.195343 -15.793291  7.564030e-03   
0.001723    5     -16.204650   0.010854 -16.193796 -15.793564  9.568644e-03   
0.002417    6     -16.204980   0.012883 -16.192097 -15.793916  1.178500e-02   
0.003391    7     -16.205400   0.015157 -16.190242 -15.794371  1.412992e-02   
0.004757    8     -16.205937   0.017722 -16.188215 -15.794963  1.659067e-02   
0.006672    9     -16.206633   0.020647 -16.185986 -15.795739  1.929381e-02   
0.009359    10    -16.207546   0.024034 -16.183512 -15.796767  2.248629e-02   
0.013128    11    -16.208765   0.028012 -16.180753 -15.798142  2.642082e-02   
0.018415    12    -16.210427   0.032746 -16.177681 -15.800025  3.125528e-02   
0.025830    13    -16.212761   0.038427 -16.174334 -15.802655  3.705632e-02   
0.036232    14    -16.216229   0.045278 -16.170951 -15.806520  4.387021e-02   
0.050823    15    -16.222219   0.053688 -16.168530 -15.812953  5.202005e-02   
0.071291    16    -16.231909   0.065754 -16.166155 -15.823078  6.368508e-02   
0.100000    17    -16.245093   0.084902 -16.160190 -15.836716  8.263472e-02   
0.140271    18    -16.261776   0.113782 -16.147993 -15.853825  1.119615e-01   
0.196760    19    -16.282481   0.155756 -16.126726 -15.874789  1.553736e-01   
0.275997    20    -16.307307   0.216568 -16.090738 -15.899516  2.181049e-01   
0.387100    21    -16.334835   0.305058 -16.029777 -15.926424  3.065338e-01   
0.543100    22    -16.357655   0.434792 -15.922863 -15.948274  4.313693e-01   
0.761700    23    -16.349459   0.620277 -15.729181 -15.940061  6.086958e-01   
1.068500    24    -16.297891   0.845297 -15.452594 -15.889997  8.299492e-01   
1.498800    25    -16.213209   1.089805 -15.123404 -15.808235  1.075146e+00   
2.102400    26    -16.092153   1.360590 -14.731563 -15.691145  1.337386e+00   
2.949000    27    -15.942196   1.655057 -14.287140 -15.545461  1.609582e+00   
4.136600    28    -15.763453   1.988317 -13.775136 -15.369436  1.939413e+00   
5.802500    29    -15.550326   2.379320 -13.171005 -15.158251  2.358701e+00   
8.139200    30    -15.296744   2.843489 -12.453255 -14.903643  2.850278e+00   
11.417000   31    -14.996463   3.390036 -11.606427 -14.599129  3.399645e+00   
16.014700   32    -14.643555   4.017419 -10.626136 -14.237535  4.033104e+00   
22.464000   33    -14.232244   4.708979  -9.523264 -13.816084  4.693510e+00   
31.510500   34    -13.747878   5.426869  -8.321009 -13.318283  5.365718e+00   
44.200100   35    -13.101428   6.094935  -7.006492 -12.655611  6.106787e+00   
62.000000   36    -12.383769   6.447182  -5.936587 -11.923561  6.573026e+00   
85.775000   37    -12.145667   6.307838  -5.837828 -11.679615  6.388668e+00   
109.550000  38    -12.808543   6.156874  -6.651669 -12.384116  6.183735e+00   
133.325000  39    -14.021225   6.752925  -7.268300 -13.663090  6.842496e+00   
157.100000  40    -15.247221   8.038856  -7.208366 -14.943737  8.226591e+00   
180.875000  41    -16.470570   9.476794  -6.993776 -16.211082  9.728594e+00   
204.650000  42    -17.685005  10.975261  -6.709745 -17.459225  1.126807e+01   
228.425000  43    -18.886710  12.495819  -6.390891 -18.689801  1.281880e+01   
252.200000  44    -20.072806  14.017170  -6.055636 -19.899023  1.435674e+01   
275.975000  45    -21.241764  15.526692  -5.715072 -21.087636  1.586830e+01   
299.750000  46    -22.392717  17.016231  -5.376487 -22.256065  1.734675e+01   
323.525000  47    -23.525308  18.480246  -5.045063 -23.401690  1.878718e+01   
347.300000  48    -24.639555  19.915109  -4.724446 -24.528776  2.019320e+01

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')